In [1]:
import numpy as np
import pandas as pd
import datetime    
import torch
import torch.nn as nn
import torch.utils.data.dataloader
from neural_net import P1_Net, do_train

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.utils.validation import column_or_1d

In [2]:
df_city = pd.read_csv("city_attributes.csv")
df_humidity = pd.read_csv("humidity.csv")
df_pressure = pd.read_csv("pressure.csv")
df_temper = pd.read_csv("temperature.csv")
df_descript = pd.read_csv("weather_description.csv")
df_direction = pd.read_csv("wind_direction.csv")
df_speed = pd.read_csv("wind_speed.csv")

In [3]:
max([df_humidity.isna().sum().mean(),
df_pressure.isna().sum().mean(),
df_temper.isna().sum().mean(),
df_direction.isna().sum().mean(),
df_speed.isna().sum().mean()])

774.3513513513514

In [4]:
df_temper.describe(include='all')

,datetime,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
count,45253,44458.000000,45252.000000,44460.000000,45250.000000,45250.000000,45252.000000,45252.000000,45250.000000,45252.000000,...,45250.000000,44460.000000,45250.000000,45250.000000,44455.000000,44460.000000,44461.000000,44455.000000,44456.000000,44460.000000
unique,45253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,2012-10-01 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,283.862654,284.992929,288.155821,284.409626,290.846116,290.215044,292.424887,295.493358,285.617856,...,285.374168,285.400406,280.343010,283.779823,291.521986,294.512307,296.497276,295.266398,294.094803,293.184253
std,NaN,6.640131,7.452438,5.332862,6.547986,6.460823,5.889992,10.829522,9.916743,9.853484,...,10.242377,10.220932,11.953626,9.802499,7.821815,6.676412,8.852984,6.324566,6.304118,7.093583
min,NaN,245.150000,262.370000,272.300000,263.780000,266.503667,265.783333,260.561333,266.059000,255.042333,...,250.390000,250.774000,243.300000,249.540000,272.179000,271.049000,271.150000,271.150000,268.682000,272.974000
25%,NaN,279.160000,279.850000,284.670000,279.830000,286.380000,286.254750,283.920000,287.680000,277.970000,...,277.350636,277.370000,271.971750,276.090000,285.366623,289.450000,289.734000,290.467333,289.881833,287.524279
50%,NaN,283.450000,284.320000,287.610000,283.940000,290.530000,290.118750,292.027486,295.586667,286.120000,...,285.927583,285.870000,281.109000,284.133250,290.932667,294.900000,296.150000,294.820000,294.150000,292.996000
75%,NaN,288.600785,289.451750,291.015167,288.530000,295.080000,294.107542,300.835000,303.050000,292.835643,...,293.796000,293.760000,290.369583,291.620000,297.270000,299.800000,303.150000,299.660000,298.930000,299.150000


In [5]:
def group_by_days(df):
    df["date"] = pd.to_datetime(df["datetime"]).dt.date
    df = df.drop('datetime', axis=1)
    df = df.groupby(df["date"]).mean()
    return df

def group_by_days_descript(df):
    df["date"] = pd.to_datetime(df["datetime"]).dt.date
    df = df.fillna('no data')
    df = df.drop('datetime', axis=1)
    df = df.groupby(df["date"]).agg(lambda x: pd.Series.mode(x)[0])
    return df

In [6]:
df_humidity = group_by_days(df_humidity)
df_pressure = group_by_days(df_pressure)
df_temper = group_by_days(df_temper)
df_descript = group_by_days_descript(df_descript)
df_direction = group_by_days(df_direction)
df_speed = group_by_days(df_speed)

In [7]:
encoder = LabelEncoder()
df_descript= df_descript.apply(encoder.fit_transform)

In [8]:
def create_dataframe_for_city(city_name):
    df =  pd.concat([
        df_humidity[city_name], df_pressure[city_name], df_temper[city_name], df_descript[city_name], df_direction[city_name], df_speed[city_name]
    ], axis=1)
    df.columns = ['humidity', 'pressure', 'temperature', 'description', 'wind_direction','wind_speed' ]
    return df

In [9]:
df_Portland  =  create_dataframe_for_city("Portland")
df_Portland

,humidity,pressure,temperature,description,wind_direction,wind_speed
date,,,,,,
2012-10-01,78.727273,1024.000000,282.118197,11,57.727273,0.000000
2012-10-02,65.833333,1023.583333,286.137728,11,214.041667,1.291667
2012-10-03,66.208333,1021.083333,289.599792,2,228.333333,2.625000
2012-10-04,51.166667,1022.875000,286.482500,12,206.750000,4.625000
2012-10-05,40.391304,1022.916667,288.286042,12,182.250000,3.708333
...,...,...,...,...,...,...
2017-11-26,87.625000,1004.500000,281.588333,8,153.750000,4.958333
2017-11-27,92.875000,1015.625000,279.922500,8,158.083333,1.666667
2017-11-28,88.458333,1019.791667,280.085833,8,144.166667,3.041667


In [10]:
df_speed_2 = pd.read_csv("wind_speed.csv")
df_speed_2["date"] = pd.to_datetime(df_speed_2["datetime"]).dt.date
df_speed_2 = df_speed_2.drop(columns=['datetime'])
df_speed_2 = df_speed_2.groupby('date').apply(lambda group: (group >= 6).any())
strong_wind = pd.concat([
    df_speed_2[city] for city in df_city['City']
])

def add_city_col(df, city):
    df['city'] = city
    return df
all_cities_df = pd.concat([
    add_city_col(create_dataframe_for_city(city), city) for city in df_city['City']
])
all_cities_df['strong_wind'] = strong_wind
all_cities_df['strong_wind'] = all_cities_df['strong_wind'].apply(lambda x: 1 if x else 0)
all_cities_df = all_cities_df.dropna().sort_values('date', kind='stable')

In [11]:
all_cities_df

,humidity,pressure,temperature,description,wind_direction,wind_speed,city,strong_wind
date,,,,,,,,
2012-10-01,78.727273,1024.000000,282.118197,11,57.727273,0.000000,Portland,0
2012-10-01,83.000000,1009.727273,289.416642,14,122.363636,1.636364,San Francisco,0
2012-10-01,78.000000,1030.000000,281.767262,1,32.272727,0.000000,Seattle,0
2012-10-01,88.000000,1013.000000,291.846501,11,0.000000,0.000000,Los Angeles,0
2012-10-01,79.909091,1013.000000,291.573495,11,0.000000,0.000000,San Diego,0
...,...,...,...,...,...,...,...,...
2017-11-30,42.000000,1025.000000,279.190000,12,360.000000,2.000000,Pittsburgh,0
2017-11-30,60.000000,1027.000000,274.510000,17,330.000000,3.000000,Toronto,0
2017-11-30,32.000000,1024.000000,283.420000,11,360.000000,4.000000,Philadelphia,0


# Przewidywanie temperatury

In [12]:
def extract_x_y_temperature(df):
    X = df.iloc[:-2].copy()
    y =  df['temperature'].iloc[4:] 

    X.reset_index(inplace= True)
    X = X.drop(columns = ['strong_wind'])
    X['day_of_year'] = pd.to_datetime(X['date']).dt.dayofyear
    X['month'] = pd.to_datetime(X['date']).dt.month

    # X['wind_dir_sin'] = np.sin(X['wind_direction']/360 * 2 * np.pi)
    # X['wind_dir_cos'] = np.cos(X['wind_direction']/360 * 2 * np.pi)
    # X['day_sin'] = np.sin(X['day_of_year']/365 * 2 * np.pi)
    # X['day_cos'] = np.cos(X['day_of_year']/365 * 2 * np.pi)
    # X = X.drop(columns = ['day_of_year'])
    # X = X.drop(columns = ['wind_direction'])

    column_names = X.columns.to_list()
    X = pd.concat([X.iloc[:-2].reset_index(drop=True),X.iloc[1:-1].reset_index(drop=True),X.iloc[2:].reset_index(drop=True)],axis = 1)
    X.columns = column_names + [c + '_1' for c in column_names] + [c + '_2' for c in column_names]
    if 'city_1' in X.columns:
        X = X.drop(columns=['city_1', 'city_2'])
    return X, y

In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

def gen_col_set(col_names):
    return [ final_col
        for col_name in col_names
        for final_col in  [col_name, col_name+"_1", col_name+"_2"] ]

def get_col_transformer_temp(df):
    return ColumnTransformer(
        [
            (
                "standarizer",
                StandardScaler(),
                gen_col_set(
                    ["humidity", "pressure", "temperature", "wind_speed", "wind_direction", "day_of_year"]
                ),
            ),
            (
                "one_hot_encoder", 
                OneHotEncoder(), 
                gen_col_set(
                    ["description", "month"]
                ) + (['city'] if 'city' in df.columns else [])
            ),
        ],
        remainder="passthrough",
    )

def prepare_temperature_data(df):
    groups = df.groupby('city')[df.columns].apply(extract_x_y_temperature)
    X_list = [group[0] for group in groups]
    y_list = [group[1] for group in groups]
    X, y = pd.concat(X_list), pd.concat(y_list)
    X['y'] = y.reset_index(drop=True)
    X = X.sort_values('date',kind='stable')
    y = X['y']
    X = X.drop(columns=['date','date_1','date_2', 'y'])

    X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size = 0.3, random_state = 0,shuffle=False)
    ct = get_col_transformer_temp(X_train)
    X_train = ct.fit_transform(X_train)
    X_test = ct.transform(X_test)
    return X_train, X_test, y_train, y_test

In [14]:
X_train, X_test, y_train, y_test = prepare_temperature_data(all_cities_df)
X_train.shape

(47175, 150)

In [15]:
dataset = torch.utils.data.TensorDataset(
    torch.from_numpy(X_train.toarray().astype('float32')), 
    torch.from_numpy(y_train.to_numpy().astype('float32')).unsqueeze(1)
)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=512, shuffle=False)
net = P1_Net(
    nn.Sequential(
        nn.Linear(X_train.shape[1], 256),
        nn.ReLU(),
        nn.Linear(256,256),
        nn.ReLU(),
        nn.Linear(256,256),
        nn.ReLU(),
        nn.Linear(256,1)
    )
)
print(net)

optimizer = torch.optim.Adam(net.parameters(), lr=0.0001)
loss = nn.MSELoss()

do_train(net, data_loader, optimizer, loss, 400)

P1_Net(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=150, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=1, bias=True)
  )
)
[1] loss: 80266.916
[2] loss: 64860.946
[3] loss: 13887.196
[4] loss: 869.173
[5] loss: 590.549
[6] loss: 473.474
[7] loss: 398.881
[8] loss: 343.455
[9] loss: 299.562
[10] loss: 263.811
[11] loss: 234.247
[12] loss: 209.460
[13] loss: 188.449
[14] loss: 170.477
[15] loss: 155.020
[16] loss: 141.621
[17] loss: 129.922
[18] loss: 119.646
[19] loss: 110.584
[20] loss: 102.534
[21] loss: 95.356
[22] loss: 88.912
[23] loss: 83.107
[24] loss: 77.853
[25] loss: 73.084
[26] loss: 68.748
[27] loss: 64.807
[28] loss: 61.229
[29] loss: 58.001
[30] loss: 55.101
[31] loss: 52.500
[32] loss: 50.199
[33] loss: 48.176
[34] loss: 46.428
[35] loss: 44.953
[36] loss:

In [16]:
from sklearn.metrics import mean_absolute_error
net.eval()

def valid_in_margin(y1, y2, margin):
    return 1 - np.count_nonzero(abs(y1-y2)>margin) / len(y1)
print("* train")
with torch.no_grad():
    y_pred1 = net(torch.from_numpy(X_train.toarray().astype('float32'))).numpy()
    y_pred1 = np.squeeze(y_pred1)

mae = mean_absolute_error(y_train, y_pred1)
print(f"\t* Mean absolute error: {mae}")
margin = 2
fraction = valid_in_margin(y_pred1, y_train, margin)
print(f"\t* Fraction of predictions with absolute error <= {margin}: {fraction}")

print("* test")
with torch.no_grad():
    y_pred2 = net(torch.from_numpy(X_test.toarray().astype('float32'))).numpy()
    y_pred2 = np.squeeze(y_pred2)

mae = mean_absolute_error(y_test, y_pred2)
print(f"\t* Mean absolute error: {mae}")
margin = 2
fraction = valid_in_margin(y_pred2, y_test, margin)
print(f"\t* Fraction of predictions with absolute error <= {margin}: {fraction}")

* train
	* Mean absolute error: 7.021524060770603
	* Fraction of predictions with absolute error <= 2: 0.13971383147853733
* test
	* Mean absolute error: 7.634810577571083
	* Fraction of predictions with absolute error <= 2: 0.12953162866610612


In [17]:
X_train, X_test, y_train, y_test = prepare_temperature_data(all_cities_df)
X_train.shape
dataset = torch.utils.data.TensorDataset(
    torch.from_numpy(X_train.toarray().astype('float32')), 
    torch.from_numpy(y_train.to_numpy().astype('float32')).unsqueeze(1)
)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=512, shuffle=False)
net = P1_Net(
    nn.Sequential(
        nn.Linear(X_train.shape[1], 256),
        nn.ReLU(),
         nn.Dropout(p=0.5),
        nn.Linear(256,1)
    )
)
print(net)

optimizer = torch.optim.Adam(net.parameters(), lr=0.0001)
loss = nn.MSELoss()

do_train(net, data_loader, optimizer, loss, 400)

P1_Net(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=150, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=256, out_features=1, bias=True)
  )
)
[1] loss: 80843.623
[2] loss: 80572.665
[3] loss: 80200.904
[4] loss: 79668.698
[5] loss: 78937.602
[6] loss: 77981.773
[7] loss: 76784.161
[8] loss: 75351.741
[9] loss: 73685.427
[10] loss: 71808.183
[11] loss: 69725.653
[12] loss: 67444.501
[13] loss: 64998.326
[14] loss: 62395.939
[15] loss: 59641.088
[16] loss: 56794.902
[17] loss: 53853.550
[18] loss: 50844.836
[19] loss: 47827.565
[20] loss: 44815.552
[21] loss: 41788.977
[22] loss: 38797.033
[23] loss: 35824.233
[24] loss: 32920.852
[25] loss: 30127.258
[26] loss: 27392.433
[27] loss: 24776.402
[28] loss: 22264.303
[29] loss: 19917.438
[30] loss: 17673.774
[31] loss: 15609.204
[32] loss: 13691.510
[33] loss: 11911.313
[34] loss: 10297.284
[35] loss: 8842.249
[36] loss: 7498.224
[37] loss: 6379.175
[38] loss

In [18]:
from sklearn.metrics import mean_absolute_error
net.eval()

def valid_in_margin(y1, y2, margin):
    return 1 - np.count_nonzero(abs(y1-y2)>margin) / len(y1)
print("* train")
with torch.no_grad():
    y_pred1 = net(torch.from_numpy(X_train.toarray().astype('float32'))).numpy()
    y_pred1 = np.squeeze(y_pred1)

mae = mean_absolute_error(y_train, y_pred1)
print(f"\t* Mean absolute error: {mae}")
margin = 2
fraction = valid_in_margin(y_pred1, y_train, margin)
print(f"\t* Fraction of predictions with absolute error <= {margin}: {fraction}")

print("* test")
with torch.no_grad():
    y_pred2 = net(torch.from_numpy(X_test.toarray().astype('float32'))).numpy()
    y_pred2 = np.squeeze(y_pred2)

mae = mean_absolute_error(y_test, y_pred2)
print(f"\t* Mean absolute error: {mae}")
margin = 2
fraction = valid_in_margin(y_pred2, y_test, margin)
print(f"\t* Fraction of predictions with absolute error <= {margin}: {fraction}")

* train
	* Mean absolute error: 3.1771367505403685
	* Fraction of predictions with absolute error <= 2: 0.3767037625861155
* test
	* Mean absolute error: 3.6946716206128265
	* Fraction of predictions with absolute error <= 2: 0.30426826252534744


In [19]:
X_train, X_test, y_train, y_test = prepare_temperature_data(all_cities_df)
X_train.shape
dataset = torch.utils.data.TensorDataset(
    torch.from_numpy(X_train.toarray().astype('float32')), 
    torch.from_numpy(y_train.to_numpy().astype('float32')).unsqueeze(1)
)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=512, shuffle=False)
net = P1_Net(
    nn.Sequential(
        nn.Linear(X_train.shape[1], 256),
        nn.ReLU(),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64,1)
    )
)
print(net)

optimizer = torch.optim.Adam(net.parameters(), lr=0.0001)
loss = nn.MSELoss()

do_train(net, data_loader, optimizer, loss, 400)

P1_Net(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=150, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=1, bias=True)
  )
)
[1] loss: 80897.110
[2] loss: 78802.257
[3] loss: 66599.003
[4] loss: 34937.756
[5] loss: 5594.957
[6] loss: 456.685
[7] loss: 326.158
[8] loss: 270.970
[9] loss: 235.794
[10] loss: 209.604
[11] loss: 188.562
[12] loss: 170.939
[13] loss: 155.824
[14] loss: 142.667
[15] loss: 131.101
[16] loss: 120.856
[17] loss: 111.721
[18] loss: 103.547
[19] loss: 96.195
[20] loss: 89.571
[21] loss: 83.589
[22] loss: 78.188
[23] loss: 73.295
[24] loss: 68.856
[25] loss: 64.828
[26] loss: 61.174
[27] loss: 57.860
[28] loss: 54.852
[29] loss: 52.123
[30] loss: 49.645
[31] loss: 47.389
[32] loss: 45.335
[33] loss: 43.466
[34] loss: 41.757
[35] loss: 40.190
[36] loss: 

In [20]:
from sklearn.metrics import mean_absolute_error
net.eval()

def valid_in_margin(y1, y2, margin):
    return 1 - np.count_nonzero(abs(y1-y2)>margin) / len(y1)
print("* train")
with torch.no_grad():
    y_pred1 = net(torch.from_numpy(X_train.toarray().astype('float32'))).numpy()
    y_pred1 = np.squeeze(y_pred1)

mae = mean_absolute_error(y_train, y_pred1)
print(f"\t* Mean absolute error: {mae}")
margin = 2
fraction = valid_in_margin(y_pred1, y_train, margin)
print(f"\t* Fraction of predictions with absolute error <= {margin}: {fraction}")

print("* test")
with torch.no_grad():
    y_pred2 = net(torch.from_numpy(X_test.toarray().astype('float32'))).numpy()
    y_pred2 = np.squeeze(y_pred2)

mae = mean_absolute_error(y_test, y_pred2)
print(f"\t* Mean absolute error: {mae}")
margin = 2
fraction = valid_in_margin(y_pred2, y_test, margin)
print(f"\t* Fraction of predictions with absolute error <= {margin}: {fraction}")

* train
	* Mean absolute error: 4.862332956343701
	* Fraction of predictions with absolute error <= 2: 0.23249602543720194
* test
	* Mean absolute error: 5.384959335249013
	* Fraction of predictions with absolute error <= 2: 0.19872397250111284


## Przewidywanie siły wiatru

In [21]:
def extract_x_y_wind(df):
    X = df.copy()
    X = X.iloc[:-2]
    y = df['strong_wind'].iloc[4:] 

    X.reset_index(inplace= True)
    X['day_of_year'] = pd.to_datetime(X['date']).dt.dayofyear
    X['month'] = pd.to_datetime(X['date']).dt.month
    # X['day_sin'] = np.sin(X['day_of_year']/365 * 2 * np.pi)
    # X['day_cos'] = np.cos(X['day_of_year']/365 * 2 * np.pi)
    # X = X.drop(columns = ['day_of_year'])


    column_names = X.columns.to_list()
    X = pd.concat([X.iloc[:-2].reset_index(drop=True),X.iloc[1:-1].reset_index(drop=True),X.iloc[2:].reset_index(drop=True)],axis = 1)
    X.columns = column_names + [c + '_1' for c in column_names] + [c + '_2' for c in column_names]
    if 'city_1' in X.columns:
        X = X.drop(columns=['city_1', 'city_2'])
    return X, y

In [22]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

def get_col_transformer_wind(df):
    return ColumnTransformer(
        [
            (
                "standarizer",
                StandardScaler(),
                gen_col_set(
                    ["humidity", "pressure", "temperature", "wind_speed", "wind_direction", "day_of_year"]
                ),
            ),
            (
                "one_hot_encoder", 
                OneHotEncoder(), 
                gen_col_set(
                    ["description", "month"]
                ) + (['city'] if 'city' in df.columns else [])
            ),
            ('pass', 'passthrough', gen_col_set(['strong_wind'])),
        ],
        remainder="passthrough",
    )

def prepare_wind_data(df):
    groups = df.groupby('city')[df.columns].apply(extract_x_y_wind)
    X_list = [group[0] for group in groups]
    y_list = [group[1] for group in groups]
    X, y = pd.concat(X_list), pd.concat(y_list)
    X['y'] = y.reset_index(drop=True)
    X = X.sort_values('date',kind='stable')
    y = X['y']
    X = X.drop(columns=['date','date_1','date_2', 'y'])

    X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size = 0.3, random_state = 0,shuffle=False)
    ct = get_col_transformer_wind(X_train)
    X_train = ct.fit_transform(X_train)
    X_test = ct.transform(X_test)
    return X_train, X_test, y_train, y_test

In [23]:
X_train, X_test, y_train, y_test = prepare_wind_data(all_cities_df)
X_train.shape

(47175, 153)

In [24]:
y_0 = np.array([1 if i==0 else 0 for i in y_train])
y_t = np.concatenate([np.expand_dims(y_0, axis=1),np.expand_dims(y_train, axis=1)],axis=1)
dataset = torch.utils.data.TensorDataset(
    torch.from_numpy(X_train.toarray().astype('float32')), 
    torch.from_numpy(y_t.astype('float32'))
)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=512, shuffle=False)
net = P1_Net(
    nn.Sequential(
        nn.Linear(X_train.shape[1], 256),
        nn.ReLU(),
        nn.Linear(256,256),
        nn.ReLU(),
        nn.Linear(256,256),
        nn.ReLU(),
        nn.Linear(256,2),
        nn.Softmax(dim=1)
    )
)
print(net)

optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
loss = nn.CrossEntropyLoss()

do_train(net, data_loader, optimizer, loss, 200)

P1_Net(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=153, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=2, bias=True)
    (7): Softmax(dim=1)
  )
)
[1] loss: 0.709
[2] loss: 0.692
[3] loss: 0.665
[4] loss: 0.633
[5] loss: 0.619
[6] loss: 0.613
[7] loss: 0.608
[8] loss: 0.604
[9] loss: 0.600
[10] loss: 0.598
[11] loss: 0.593
[12] loss: 0.590
[13] loss: 0.587
[14] loss: 0.582
[15] loss: 0.582
[16] loss: 0.579
[17] loss: 0.583
[18] loss: 0.575
[19] loss: 0.572
[20] loss: 0.565
[21] loss: 0.561
[22] loss: 0.561
[23] loss: 0.560
[24] loss: 0.558
[25] loss: 0.556
[26] loss: 0.549
[27] loss: 0.554
[28] loss: 0.543
[29] loss: 0.542
[30] loss: 0.537
[31] loss: 0.542
[32] loss: 0.538
[33] loss: 0.529
[34] loss: 0.521
[35] loss: 0.527
[36] loss: 0.527
[37] loss: 0.533
[38] loss: 0.

In [25]:
from sklearn.metrics import roc_auc_score
net.eval()
print("* train")
with torch.no_grad():
    y_pred1 = net(torch.from_numpy(X_train.toarray().astype('float32')))[:,1]
print(f"\t* ROC_AUC: {roc_auc_score(y_train, y_pred1)}")

print("* test")
with torch.no_grad():
    y_pred1 = net(torch.from_numpy(X_test.toarray().astype('float32')))[:,1]
print(f"\t* ROC_AUC: {roc_auc_score(y_test, y_pred1)}")

* train
	* ROC_AUC: 0.8958818590728733
* test
	* ROC_AUC: 0.6315882784791245


In [26]:
X_train, X_test, y_train, y_test = prepare_wind_data(all_cities_df)
X_train.shape
y_0 = np.array([1 if i==0 else 0 for i in y_train])
y_t = np.concatenate([np.expand_dims(y_0, axis=1),np.expand_dims(y_train, axis=1)],axis=1)
dataset = torch.utils.data.TensorDataset(
    torch.from_numpy(X_train.toarray().astype('float32')), 
    torch.from_numpy(y_t.astype('float32'))
)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=512, shuffle=False)
net = P1_Net(
    nn.Sequential(
        nn.Linear(X_train.shape[1], 256),
        nn.ReLU(),
        nn.Dropout(p=0.5),
        nn.Linear(256,2),
        nn.Softmax(dim=1)
    )
)
print(net)

optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
loss = nn.CrossEntropyLoss()

do_train(net, data_loader, optimizer, loss, 200)

P1_Net(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=153, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=256, out_features=2, bias=True)
    (4): Softmax(dim=1)
  )
)
[1] loss: 0.696
[2] loss: 0.654
[3] loss: 0.638
[4] loss: 0.629
[5] loss: 0.622
[6] loss: 0.617
[7] loss: 0.612
[8] loss: 0.610
[9] loss: 0.607
[10] loss: 0.605
[11] loss: 0.604
[12] loss: 0.602
[13] loss: 0.601
[14] loss: 0.600
[15] loss: 0.599
[16] loss: 0.597
[17] loss: 0.597
[18] loss: 0.595
[19] loss: 0.595
[20] loss: 0.593
[21] loss: 0.592
[22] loss: 0.592
[23] loss: 0.590
[24] loss: 0.590
[25] loss: 0.587
[26] loss: 0.587
[27] loss: 0.586
[28] loss: 0.585
[29] loss: 0.584
[30] loss: 0.583
[31] loss: 0.582
[32] loss: 0.581
[33] loss: 0.580
[34] loss: 0.579
[35] loss: 0.579
[36] loss: 0.578
[37] loss: 0.576
[38] loss: 0.576
[39] loss: 0.575
[40] loss: 0.574
[41] loss: 0.574
[42] loss: 0.572
[43] loss: 0.572
[44] loss: 0.571
[45] loss: 

In [27]:
from sklearn.metrics import roc_auc_score
net.eval()
print("* train")
with torch.no_grad():
    y_pred1 = net(torch.from_numpy(X_train.toarray().astype('float32')))[:,1]
print(f"\t* ROC_AUC: {roc_auc_score(y_train, y_pred1)}")

print("* test")
with torch.no_grad():
    y_pred1 = net(torch.from_numpy(X_test.toarray().astype('float32')))[:,1]
print(f"\t* ROC_AUC: {roc_auc_score(y_test, y_pred1)}")

* train
	* ROC_AUC: 0.8472582955958052
* test
	* ROC_AUC: 0.6839878636916268


In [28]:
X_train, X_test, y_train, y_test = prepare_wind_data(all_cities_df)
X_train.shape
y_0 = np.array([1 if i==0 else 0 for i in y_train])
y_t = np.concatenate([np.expand_dims(y_0, axis=1),np.expand_dims(y_train, axis=1)],axis=1)
dataset = torch.utils.data.TensorDataset(
    torch.from_numpy(X_train.toarray().astype('float32')), 
    torch.from_numpy(y_t.astype('float32'))
)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=512, shuffle=False)
net = P1_Net(
    nn.Sequential(
        nn.Linear(X_train.shape[1], 256),
        nn.ReLU(),
        nn.Linear(256,128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,2),
        nn.Softmax(dim=1)
    )
)
print(net)

optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
loss = nn.CrossEntropyLoss()

do_train(net, data_loader, optimizer, loss, 200)

P1_Net(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=153, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=2, bias=True)
    (7): Softmax(dim=1)
  )
)
[1] loss: 0.703
[2] loss: 0.688
[3] loss: 0.660
[4] loss: 0.633
[5] loss: 0.618
[6] loss: 0.612
[7] loss: 0.608
[8] loss: 0.605
[9] loss: 0.602
[10] loss: 0.599
[11] loss: 0.597
[12] loss: 0.595
[13] loss: 0.592
[14] loss: 0.589
[15] loss: 0.592
[16] loss: 0.587
[17] loss: 0.582
[18] loss: 0.581
[19] loss: 0.584
[20] loss: 0.577
[21] loss: 0.573
[22] loss: 0.573
[23] loss: 0.569
[24] loss: 0.576
[25] loss: 0.568
[26] loss: 0.562
[27] loss: 0.559
[28] loss: 0.556
[29] loss: 0.552
[30] loss: 0.550
[31] loss: 0.545
[32] loss: 0.543
[33] loss: 0.545
[34] loss: 0.561
[35] loss: 0.543
[36] loss: 0.536
[37] loss: 0.527
[38] loss: 0.53

In [29]:
from sklearn.metrics import roc_auc_score
net.eval()
print("* train")
with torch.no_grad():
    y_pred1 = net(torch.from_numpy(X_train.toarray().astype('float32')))[:,1]
print(f"\t* ROC_AUC: {roc_auc_score(y_train, y_pred1)}")

print("* test")
with torch.no_grad():
    y_pred1 = net(torch.from_numpy(X_test.toarray().astype('float32')))[:,1]
print(f"\t* ROC_AUC: {roc_auc_score(y_test, y_pred1)}")

* train
	* ROC_AUC: 0.887167637953014
* test
	* ROC_AUC: 0.628390203745488
